# Task 1


We implemented the unknown a_i with help of the three different ways. The a looks exactly the same with the normal equation, the qr factorization and with the svd decomposition: a= [  3.99977426e+00,  -1.53545576e-03 ,  3.00267362e+00, 1.99819673e+00, 1.00040411e+00]. If we plot the discrete values and the function it fits really well. 


To compare the execution time we can firstly note that we have to build A (the matrix with the function in it) for all methods. So this time doesn't matter. 
For the normal equation solution we need to transpose A, built two products of two matrices and solve one linear equation system.  (If we restart the kernel, it takes about 378ms for the execution)
For the qr factorization we need the qr factorization command or compute it and we need also one transpose, one product and one solution of a linear equation system. (about 344ms)
For the last solution, the svd decomposition, we need the svd decomposition, two inverses, three products and build the diagonal matrix with the diagonal entries from the svd decomposition. (123ms)
It seems that the svd takes the most time because there are the most calculations. But even when executing the program with the existing A and the stored solution the normal equation takes the most time (about 20ms). 
We think the execution time of the last method is so good, because we don't have to use the solve command. We can built the inverse of the diagonal matrix very easy so that we can use $x=A^{-1}*b$ to solve the linear equation. 

You can find the code in the attachment also with the plot.




# Task 2

For the problem $A \mapsto A^{-1}b$, with a given matrix $A$ and b as input, we have that the condition number 

$$
\frac{||\delta x||}{||x||}\frac{||b||}{||\delta b||} ≤ \kappa(A) = ||A||||A^{-1}|| = \frac{\sigma_{n}}{\sigma_{1}}
$$

when the norm is the 2-norm. Then, if we choose $b=u_{1}$, where $u_{1}$ is the left singular vector corresponding to the greatest singular value $\sigma_{1}$ in te SVD-decomposition of A,

$$
x = A^{-1}b = V \Sigma^{-1} U^{T} u_{1} = V \Sigma^{-1}
\begin{bmatrix}
    u_{1}^{T}    \\
    \vdots       \\
    u_{n}^{T}             
\end{bmatrix} u_{1} = V 
\begin{bmatrix}
    \frac{1}{\sigma_{1}} & \ldots  & 0  \\
    \vdots & \ddots & \vdots      \\
    0 & \ldots & \frac{1}{\sigma_{n}}           
\end{bmatrix} 
\begin{bmatrix}
    1    \\
    0    \\
    \vdots       \\
    0             
\end{bmatrix} =
\begin{bmatrix}
    v_{1} & \ldots & v_{n}             
\end{bmatrix}
\begin{bmatrix}
    \frac{1}{\sigma_{1}} \\
    0 \\
    \vdots       \\
    0             
\end{bmatrix} = \frac{v_{1}}{\sigma_{1}},
$$


where $v_{1}$ is the corresponding right singular vector. 
Then

$$
\frac{||b||}{||x||} = \frac{||u_{1}||}{||\frac{1}{\sigma_{1}}v_{1}||}=\sigma_{1}.
$$

Similarily, with the choice $\delta b = u_{n}$ – the left singular vector associated with the smallest singular value $\sigma_{n}$, 

$$
\frac{||\delta b||}{||\delta x||} = \frac{||\frac{v_{1}}{\sigma_{n}}||}{||u_{n}||}=\frac{1}{\sigma_{n}}.
$$

Finally

$$
\frac{||\delta x||}{||x||}\frac{||b||}{||\delta b||} = \frac{\sigma_{1}}{\sigma_{n}} = ||A||||A^{-1}||, 
$$

and the equality holds.

# Task 3

We constructed the Hilbert matrix H of dimension 50 and its inverse in python. The condition number of the matrix we determined to

$$
\kappa(H) = ||H||||H^{-1}|| = 1.42\cdot 10^{74}.
$$

Our result of task 2 suggests that we should obtain this number by computing

$$
\frac{||\delta x||}{||x||}\frac{||b||}{||\delta b||},
$$

using c as the left singular vectors described in task 2. We can get these by SVD-decomposing H,

$$
H = U \Sigma V^{T},
$$

and taking $u_{1}, u_{n}$ from the matrix U. This works well for small dimensions (up to 10), so that the equality from task 2 holds. However for larger dimensions (and specifically for n=50), the condition number is smaller than $||H||||H^{-1}||$, for n=50: 

$$
\frac{||\delta x||}{||x||}\frac{||b||}{||\delta b||} = 1.74 \cdot 10^{16}.
$$

We cannot explain why we dont obtain the equality, but we suspect it can have to do with the properties of H (since it is very ill conditioned small numerical perturbations affects gravely the solution?). In particular since it works for lower dimensions we dont suspect an error in our implementation.

We tried to put in other vectors for $b, \delta b$ and saw that we never suceeded to get a larger condition number for this problem, which motivates that the choice was the good one.

# Attachements

In [ ]:
import numpy as np
import scipy.linalg as sl
import csv
import matplotlib.pyplot as plt

# import the text file and create two vectors v and t

import os

try:
	datfilestr = input('Which file name?\n---->  ')
	datfile=open(datfilestr,'r')
except FileNotFoundError:
	txt="""File {} not found in catalogue {}. 
	    Make sure you downloaded the file from the course homepage
	    and you run this script in its catalogue.
	    """
	print(txt.format(datfilestr,os.path.abspath(os.curdir)))
else:
	line = [ll.strip('\n').split(',') for ll in datfile.readlines()]
	t,v=list(zip(*[(float(l[0]),float(l[1])) for l in line]))
	t=np.array(t)
	v=np.array(v)
	print('You get now two shape {sh} arrays "t" and "v" with data.'.format(sh=t.shape))

In [ ]:
# blue points are the given diskret values 
# implement a code which uses the least square problem with 3 different 
class Least_square_methode():
    
    def __init__(self, v, t):
        self.v = v
        self.t = t
        self.A = np.empty((0,5), int)
        for t_i in self.t: 
            self.A = np.vstack([self.A,[1, t_i, t_i**2, t_i**3, t_i**4]])
            
    def normal_equation(self): 
        AT = np.transpose(self.A)
        normal = sl.solve(np.dot(AT, self.A), np.dot(AT,v)) 
        return normal
    def qr_factorisation(self):
        q,r = np.linalg.qr(self.A) # compute the reduced QR factorization
        qT = np.transpose(q) # build the transpose because q isn't square
        qv = np.dot(qT,v) 
        beta = sl.solve(r,qv) # solve the upper-triangular system R*x = QT*v  beta is the x vector we are searching for
        return beta
    def svd_decomposition(self):
        u,s,d = np.linalg.svd(self.A) # computes the svd
        u = np.linalg.inv(u)
        uv = np.dot(u,self.v)
        S = np.zeros((5, 80))
        for i in range(5):
            S[i,i] = 1/s[i]
        w = np.dot(S,uv)
        gamma = np.dot(np.linalg.inv(d),w)
        return gamma
    
A = Least_square_methode(v,t)
normal = A.normal_equation()
print(normal)
qr = A.qr_factorisation()
print(qr)
svd = A.svd_decomposition()
print(svd)
g = np.arange(0,2,0.01)
plt.plot(g, ((svd[0]+svd[1]*g+svd[2]*g**2+svd[3]*g**3+svd[4]*g**4)),'r-', t,v,'.', g, ((qr[0]+qr[1]*g+qr[2]*g**2+qr[3]*g**3+qr[4]*g**4)),'g-')
plt.show()